In [1]:
import models
import pandas as pd

In [2]:
# METRIC = 'F1'
# ALGO = 'detection_correction'
# # ALGO = 'correction'
MODEL = 'CNN_Attention_256_filters_3_kernels'
COMMODITY = 'copper_over'
confidence_levels = [0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 0.95]

algo = ['correction', 'detection_correction']

In [3]:
# df = models.npy_to_bowpy(f'{COMMODITY}/test/predictions/test/{MODEL}_predictions.csv', f'copper/test/predictions/test_{METRIC}_{ALGO}/', confidence_levels)
df = models.npy_to_bowpy(f'{COMMODITY}/test/predictions/test/{MODEL}_predictions.csv', f'{COMMODITY}/test/predictions/', confidence_levels, algo[1])
df

,pred,corr,true_positive,false_positive,test_F1_detection_correction_rule_result_conf_0.55,test_F1_detection_correction_rule_result_conf_0.5,test_F1_detection_correction_rule_result_conf_0.6,test_F1_detection_correction_rule_result_conf_0.7,test_F1_detection_correction_rule_result_conf_0.8,test_F1_detection_correction_rule_result_conf_0.95,...,test_Recall_detection_correction_rule_result_conf_0.8,test_Recall_detection_correction_rule_result_conf_0.95,test_Recall_detection_correction_rule_result_conf_0.9,test_Precision_detection_correction_rule_result_conf_0.55,test_Precision_detection_correction_rule_result_conf_0.5,test_Precision_detection_correction_rule_result_conf_0.6,test_Precision_detection_correction_rule_result_conf_0.7,test_Precision_detection_correction_rule_result_conf_0.8,test_Precision_detection_correction_rule_result_conf_0.95,test_Precision_detection_correction_rule_result_conf_0.9
0,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,1,0,0,1,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
218,1,0,0,1,1,1,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
219,1,0,0,1,1,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
220,1,0,0,1,1,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [4]:
def calculate_metrics(df, rule_result_column):
    TP = ((df[rule_result_column] == 1) & (df['corr'] == 1)).sum()
    FP = ((df[rule_result_column] == 1) & (df['corr'] == 0)).sum()
    FN = ((df[rule_result_column] == 0) & (df['corr'] == 1)).sum()
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return f1, recall, precision

In [5]:
f1, recall, precision = calculate_metrics(df, "pred")
print(f'Base model results\nf1: {f1}\nrecall: {recall}\nprecision: {precision}')

Base model results
f1: 0.25263157894736843
recall: 0.6857142857142857
precision: 0.15483870967741936


In [6]:
df.to_numpy().dump('edcr/data/test.npy')